In [6]:
from langgraph.graph import StateGraph,START,END
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
import yfinance as yf
from datetime import datetime
from typing import TypedDict
import time
import re

# workflow simple design
[Trigger]</br>
     ↓</br>
[Discovery Layer] </br>
     ↓ </br>
[Pre-Filter Layer] </br>
     ↓ </br>
[Deep Analysis Layer] </br>
     ↓ </br>
[Risk Management Layer]  </br>
     ↓ </br>
[Portfolio Agent] </br>
     ↓ </br>
BUY? (with confidence score) </br>
     ↓ </br>
[Alert Agent] </br>
     ↓ </br>
⏸ WAIT FOR HUMAN (with timeout) </br>
     ↓ </br>
YES ──→ [Execution Engine] → [Broker] </br>
NO  ──→ [Log & End] → [Feedback to Discovery] </br>
     ↓ </br>
[Memory Agent] ← Store execution data </br>
     ↓ </br>
[Performance Analytics] ← New component </br>
     ↓ </br>
[Confirmation Agent] </br>
     ↓ </br>
[Final Report Agent] </br>

In [ ]:
model = ChatGroq(model = 'llama-3.1-8b-instant',temperature=0.3, max_tokens = 500)
search_tool = TavilySearch(max_results = 5)

In [8]:
class StockState(TypedDict):
    trigger_type: str
    trigger_source: str
    trigger_time: str
    query: str

Trigger for firing agentic search and for buy and sale

In [10]:
def scheduled_triigger(state:StockState):
    now = datetime.now()
    hour = now.hour
    if 9 <=hour <11:
        session="market open"
        query = "Top gaining stocks at market open today US"
    elif 11 <=hour<14:
        session = "mid day"
        query = "Top momentum stocks midday trading US market"
    elif 14<=hour <16:
        session = 'pre close'
        query = "Top stocls gaining volume per-market close US"
    else:
        session = "after-hurs"
        query   = "After hours top gaining stocks US market today"
    
    print(f" Scheduled Trigger fired - session: {session}")
    return{
        "trigger_type": "scheduled",
        "trigger_source": f"corn job - {session} session",
        "trigger_time": now.strftime("%Y-%m-%d %H:%M:%S"),
        "query": query
    }
def manual_trigger(query:str):
    "for manual query and use  when user want specific search"
    print(f" manual trigger fired - query: '{query}'")
    return{
        "trigger_type": "manual",
        "trigger_source": "user input",
        "trigger_time" : datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "query": query
    }